# 1. Import Libraries ✅

<div style="font-family:verdana;line-height:2rem;">
    <ol>
        <li>Here we import the necessary libraries for running the submission file</li>
        <li>The optional imports are mentioned at the bottom</li>
        <li>Primarily following are our dependencies</li>
        <ul>
            <li>sklearn (machine learning models)  🚀</li>
            <li>numpy (mathmatical operations on multidimensional arrays)   💐</li>
            <li>pandas (handling dataframes)  💿</li>
            <li>matplotlib (plotting)  📈</li>
            <li>lightgbm and xgboost (Gradient Boosting Models)  💪</li>
            <li>sklearnex (Intel's optimization library for scikit-learn)  ⏰</li>
         </ul>
    </ol>
</div>

In [1]:
import gc
import os
import joblib
from pathlib import Path

import numpy as np
import pandas as pd

from tqdm import tqdm, trange

from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import matplotlib.pyplot as plt
import seaborn as sns

from sklearnex import patch_sklearn
patch_sklearn()

%matplotlib inline
tqdm.pandas()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


# 2. Important Files

<div style="font-family:verdana;line-height:2rem;">
    Following the the primary important files that we will be using during the training session for this notebook
    <ol>
        <li> <code>gee_features_10pct.csv:</code> This file contains all the extracted features friom the google earth engine and the DHS Survey data. The <b>10pct</b> data is a smaller version of the main data where only 10 percent of the data is used</li>
        <li><code>training_labels.csv:</code> Contains the training labels for predicting and training our maching learning model. It can also be indexed by <b>DHSID</b></li>
        <li><code>sample submission.csv:</code> Contains the Id of all the entities for which the prediction needs to be done</li>
    </ol>
</div>

In [2]:
GEE_FEATURES = '/kaggle/input/maternal-and-child-health-monitoring-in-lmics/gee_features_10pct.csv'
TRAIN_FILE = '/kaggle/input/maternal-and-child-health-monitoring-in-lmics/training_label.csv'
TEST_FILE = '/kaggle/input/maternal-and-child-health-monitoring-in-lmics/sample submission.csv'

# 3. Data Preprocessing

<div style="font-family:verdana;line-height:2rem;">
    In the following section we preocess the training data first for easier ingestion into the machine learning models like those provided by <code>scikit-learn</code>. Following steps are done in order.
    <ol>
        <li>All the three tables described in the previous section are first set to index with the <code>DHSID</code> parameter</li>
        <li>Drop all the columns which contain at least one <code>NaN</code> value as these columns are usually >= 50% empty and do not provide much data even after imputation</li>
        <li>Drop the columns which contain entitrely distinct values like in case of <code>new_ind</code> field as it does not provide us with any new information that will be useful for predicting</li>
        <li>Remove th duplicated index and keep only the last row. In case of duplicated items we only mainitan the latest entry of the survey and forego the prior entries</li>
        <li>One Hot Encode the categorical columns as it converts the textual labels to numerical data easier for ingestion by the machine learning model</li>
        <li>Drop the columns from the <code>training_labels</code> which are already existing in the <code>gee_features</code> as they do not provide any new information</li>
    </ol>
</div>

In [3]:
# Load the features from gee_features
df = pd.read_csv(GEE_FEATURES).set_index('DHSID')

# Drop the columns containing any nil values
df.dropna(inplace=True, axis='columns')

# new_ind contains completely distinct values, hence not of much use
df.drop('new_ind', axis='columns', inplace=True)

# Drop the rows with duplicate indexes and keep the latest survey results for them
df = df[~df.index.duplicated(keep='last')]

/tmp/ipykernel_32/284820401.py:2: DtypeWarning: Columns (6,7,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(GEE_FEATURES).set_index('DHSID')


# 5. Data Visualization 📈

<div style="font-family:verdana;line-height:2rem;">
    Following cell represents the distribution of the data over the different parts of the world. We use the <span style="color:crimson;">latitude</span> and the <span style="color:crimson;">longitude</span> data for plotting on the world map. For generating the world map, we make use of the utilities provided by <code>Plotly</code>
</div>

In [4]:
import plotly.express as px

fig = px.scatter_geo(df[['LATNUM', 'LONGNUM']], lat = 'LATNUM', lon = 'LONGNUM', color=df['DHSCC'])
fig.update_layout(title = 'Countries', title_x = 0.5)

<div style="font-family:verdana;line-height:2rem;">
    Here we continue on with the data preprocessing steps as mentioned in Section 4. We continue on with One Hot Encoding the Categorical Columns and mpve on to dropping the repeated features from the training data.
</div>

In [5]:
# One Hot Encode the categorical columns
cat_cols = ['DATUM', 'DHSCC', 'DHSREGNA', 'SOURCE', 'URBAN_RURA', 'key1']
cat_df = pd.get_dummies(df[cat_cols])
df.drop(cat_cols, axis='columns', inplace=True)
gc.collect()

# create the Feature DataFram by joining the encoded categorical columns
df = df.join(cat_df, on='DHSID')

In [6]:
# Load thre training and testing labels
train = pd.read_csv(TRAIN_FILE).set_index('DHSID')
test  = pd.read_csv(TEST_FILE).set_index('DHSID')

# Drop the columns which are duplicate in train and features
train.drop(['DHSYEAR', 'DHSCLUST', 'LATNUM', 'LONGNUM', 'URBAN_RURA'], inplace=True, axis='columns')

<div style="font-family:verdana;line-height:2rem;">
    This cell consists of the data labels for which we need to generate the predictions for. There are primarily 6 <code>labels</code> on which the prediction task needs to be performed. Those labels are stored in array named <span style="color:crimson;">pred_cols</span> which will be referred later in this notebook as well
</div>

In [6]:
# Features to be predicted
pred_cols = ['Mean_BMI', 'Median_BMI', 'Unmet_Need_Rate', 
             'Under5_Mortality_Rate', 'Skilled_Birth_Attendant_Rate', 
             'Stunted_Rate']

# 6. Models and Utilities 💿

<div style="font-family:verdana;line-height:2rem;">
    Following section contains the <code>Model Class</code> and the <code>Utilities</code> that will be used for training the model and scoring the trained model. Following are the two utilities.
    <ol>
        <li><code>mcrmse:</code> For computing the columnwise mean rmse which is also used as the scoring metric for the competition</li>
        <li><code>rmse:</code> For computing the RMSE score of the data along any single column or single label</li>
    </ol>
</div>

In [8]:
def mcrmse(y, y_hat):
    return rmse(y, y_hat).mean()

def rmse(y, y_hat):
    mse = ((y - y_hat)**2).mean()
    return mse ** 0.5

<div style="font-family:verdana;line-height:2rem;">
    Since we are performing <code>Boosting on Errors</code> methodology in our notebook, we name the model class thatw ill be training and fitting on the data as the <span style="color:crimson;">BoostOnErrorEnsemble</span> class. This class is written by extending the scikit-learn's <code>BaseEstimator</code> class and the <code>TransformerMixin</code> class. the model provides <code>fit</code> function whic will be used for fitting the models on the data provided and the <code>predict</code> function to predict on any new data that the model will be encountering. Following are the layers of model that is being followed. 
    <ol>
        <li><span style="color:crimson;">RandomForestRegressor:</span> is used to build the base layer model, since it has the highest generalization capability and performs best among all the models for multi label regresssion. This provides the baseline to provide other models to build on. We will cal this <code>Level 1</code> model</li>
        <li><span style="color:crimson;">LGBMRegressor:</span> is built on th errors which the <b>Level 1</b> model was not able to capture the trend on. The light GBM model is based on Gradient Boosting Methodology, and hence has higher ability to model the error of the Random Forest model and improve the performance further. We will cal this <code>Level 2</code> model</li>
        <li><span style="color:crimson;">LGBMRegressor:</span> The <code>Level 3 and 4</code> models are built using the XGBoost Regressor model which is also a Gradient Bossting based model and is able to model errpr much striongly as compared to the previous layers of the model.</li>
    </ol>
    Finally the output of all the above layers are combined for providing the prediction value for the labels provided to use for predicting, The <code>Predict</code> function
 will return the value of all the 6 labels for which the prediction needs to be done</div>

In [9]:
class BoostOnErrorEnsemble(BaseEstimator, TransformerMixin):
    def __init__(self, pred_cols):
        self.pred_cols = pred_cols
        self.model_1 = RandomForestRegressor(n_estimators=2000,verbose=1,n_jobs=-1,random_state=42)
        self.model_2 = {pcol: LGBMRegressor(n_estimators=1000, n_jobs=-1, verbose=1, random_state=42) for pcol in pred_cols}
        self.model_3 = XGBRegressor(n_estimators=1000,verbosity=1,n_jobs=-1,random_state=42)
        self.model_4 = XGBRegressor(n_estimators=2000,verbosity=1,n_jobs=-1,random_state=42)

    def fit(self, X, y):
        if not isinstance(y, pd.DataFrame):
            y = pd.DataFrame(y, columns=self.pred_cols)
            
        non_na_cols = y.dropna().index
        self.model_1.fit(X.loc[non_na_cols], y.loc[non_na_cols])
        
        e_1 = (y - self.model_1.predict(X)).copy(deep=True)
        e_2 = e_1.copy(deep=True)
        
        for pcol in self.pred_cols:
            self.model_2[pcol].fit(X, e_1[pcol])
            e_2[pcol] -= self.model_2[pcol].predict(X)
        
        self.model_3.fit(X, e_2)
        
        e_3 = (e_2 - self.model_3.predict(X)).copy(deep=True)
        self.model_4.fit(X, e_3)
        
        return self

    def predict(self, X):
        m1_pred = self.model_1.predict(X)
        m2_preds = np.column_stack([self.model_2[pcol].predict(X) for pcol in self.pred_cols])
        m3_pred = self.model_3.predict(X)
        m4_pred = self.model_4.predict(X)
        
        combined_pred = m1_pred + m2_preds + m3_pred + m4_pred
        return pd.DataFrame(combined_pred, columns=self.pred_cols, index=X.index)


In [10]:
# Prepare the training data by joining the features with training labels
train_df = df.join(train, how='inner')

# Prepare the training data by joining the features with testing labels
test_df  = df.join(test,  how='inner')

X = train_df.drop(pred_cols, axis='columns')
y = train_df[pred_cols]

# 7. Feature Selection ⏰

<div style="font-family:verdana;line-height:2rem;">
The following cell presents the process of selecting fetaures from the <b>feature_importance_</b> field that is provided the the Random Forest model after fitting on the training data. These feature importances are used to determing which of the features among the vast array of google earth engine fetures that has been provided, is actually relevant for prediction.
</div>

<div class="alert alert-block alert-info" style="font-family:verdana;margin-top:2rem;">
    📌 Please note that the run of the following cell may take over several hours to progress (>= 2 hours) and hence it is recommended to use the already selected features from the feature file as shown in the next cell
</div>

In [ ]:
# non_nan_rows = y.dropna().index

# tX = X.loc[non_nan_rows]
# ty = y.loc[non_nan_rows]

# reg = RandomForestRegressor(n_estimators=8000,verbose=1,n_jobs=-1,random_state=42)
# reg.fit(tX, ty)

<div style="font-family:verdana;line-height:2rem;">
    <ol> 
        <li>If you have run the previous cell, the uncomment the first line which will be converting the fetaure importances into pandas frame and used thereafter</li>
        <li>If not then don't uncomment the first line, instead the line folloing will be executed which will be using the feature importance fromt he already trained model and converted to csv file for ingestion anytime later</li>
    </ol>
    Once we have the feature importance, we will be sorting them from high to low and select only the top 100 features which will help us for prediction. This is beacuse after top <code>100 features</code> the importance of the features drops significantly below 0.01 and is of not much importance for us to perform prediction task.
</div>

In [12]:
# feat_imp = pd.DataFrame(reg.feature_importance_, index=X.columns, columns=['importance'])

feat_imp = pd.read_csv('/kaggle/input/feature-importance/important_features.csv', index_col=0)

sort_imp = feat_imp.sort_values(by='importance', ascending=False).head(100)
imp_cols = sort_imp.index.values

<div style="font-family:verdana;line-height:2rem;">
    In the following cell we perform KNN Imputation of the data to fill in all the misssing values in the training data as models like Random Forest Cannot handle missing values. Gradient Boosting method does handle missing values but only when the missing values are present in the features and not in the labels. We perform <code>KNN Imputation</code> on the training labels as well since it imporves the performance of the model on the fields where the data is very less, smoothening out the labels over a wider distribution
</div>

In [15]:
dX = X[imp_cols].join(y, how='inner')
imputer = KNNImputer(n_neighbors=20)
imp_dX = imputer.fit_transform(dX)

df_X = pd.DataFrame(imp_dX[:, :-6], index=X.index, columns=imp_cols)
df_y = pd.DataFrame(imp_dX[:, -6:], index=y.index, columns=y.columns)

# 8. Training Model 🚀

<div style="font-family:verdana;line-height:2rem;">
    In this step we perform the training of the model, by calling the <code>fit</code> method of the model over the training data that has been prepared after imputation and selecting the op 100 fetures from the all available list of features 
</div>

In [16]:
model = BoostOnErrorEnsemble(pred_cols)
model.fit(df_X, df_y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   40.4s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:  7.0min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Do

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008362 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18323
[LightGBM] [Info] Number of data points in the train set: 10029, number of used features: 100
[LightGBM] [Info] Start training from score 0.003386
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009308 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18323
[LightGBM] [Info] Number of data points in the train set: 10029, number of used features: 100
[LightGBM] [Info] Start training from score 0.001976
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008277 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18323
[LightGBM] [Info] Number of data points in the train set: 10029, number of used features: 100
[LightGBM] [

BoostOnErrorEnsemble(pred_cols=['Mean_BMI', 'Median_BMI', 'Unmet_Need_Rate',
                                'Under5_Mortality_Rate',
                                'Skilled_Birth_Attendant_Rate',
                                'Stunted_Rate'])

In [17]:
mcrmse(df_y, model.predict(df_X))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:    4.2s finished


0.0016286613667387364

# 9. Inferencing

<div style="font-family:verdana;line-height:2rem;">
    In this step we will be performing the model inference. Or model is pretty fast and will inferenc on the whole testing data within 2-5 seconds and provide the output. The model provides the <code>predict</code> method that can be used for inferencing over the testing data.
</div>

In [18]:
preds = model.predict(test_df[imp_cols])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:    1.3s finished


In [19]:
preds

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
DHSID,,,,,,
ML200600000390,22.283330,21.812404,52.084618,11.450336,75.416159,28.816197
BO200800002157,25.039198,24.731913,17.005703,7.202890,60.659760,33.336486
TL201600000282,21.042890,20.428862,63.493533,7.719329,34.577372,49.624393
BF201000000006,20.569007,20.253794,61.012349,15.982218,57.137009,36.188273
NG200800000031,21.110791,20.346199,87.069509,26.270225,4.322171,55.761261
...,...,...,...,...,...,...
KE200800000234,23.614767,22.425309,35.017365,9.372234,77.275728,28.479150
KE201400001195,24.031131,23.563413,31.247320,6.039422,55.653238,34.370937
ML201200000249,21.383708,21.158974,90.547919,13.306745,10.807300,48.699362


# 10. Optional

<div style="font-family:verdana;line-height:2rem;">
    The following cell provides the code for applying AutoML on the training data, so that different compbinations of ensembling and state of the art machine learningmdoels are applied to find the best ensembling methodology for our data. We will see that BoostOn Error is a very efficient ensembling that will be generating the best results in most cases
</div>

<div class="alert alert-block alert-info" style="font-family:verdana;margin-top:2rem;">
    📌 Please note that the run of the following cell may take over several hours to progress (>= 18 hours, >= 3 hours per feature) and hence it is recommended to run the following cell on a personal machine which will support the hours of training. The following cell is NOT GPU optimized, so better run on CPU machine.
</div>



In [ ]:
!pip3 install 'mljar-supervised' 

In [ ]:
from supervised.automl import AutoML

automl_sub = pd.DataFrame()

for i, pcol in enumerate(pred_cols):
    print(f'[STARTED] ({i+1}/{len(pred_cols)}) LABEL: {pcol}')
    
    # Single out the labels of the columns on which predictions need to be performed
    y_pcol = y[pcol]
    
    # Drop the values containing NaN values on labels
    y_pcol = y_pcol[~y_pcol.isna()]
    
    # Join the labels which are not NaN with the  respective feature columns
    X_new = pd.merge(X[imp_cols], y_pcol, on='DHSID', how='inner')
    
    # Separate out the feature from the labels
    X_pcol = X_new.drop(pcol, axis = 1)
    y_pcol = X_new[pcol]
    
    # Fit the auto ml model on the column we have selected (time = 3 hours)
    automl = AutoML(mode="Compete", random_state=42, total_time_limit=3*3600)
    automl.fit(X_pcol, y_pcol)
    
    # Auto ML submission for the column is accumulated
    automl_sub[pcol] = automl.predict(test)

    print(f'[COMPLETED] ({i+1}/{len(pred_cols)}) LABEL: {pcol}')
    
    # Clean and collect garbage memory
    gc.collect()